# 머신러닝   
       
14주 1강: 앙상블 모델

이번 시간에는 작은 모델들의 예측을 모아 성능을 높이는 앙상블 모델을 돌아보겠습니다.

## 앙상블 모델
**앙상블 모델(Ensemble Model)** 은  여러 개의 개별 모델을 조합하여 최적의 모델로 일반화하는 방법을 말합니다

## 투표 분류기
### 가장 단순한 투표 분류기
* 단순하게 여러 모델을 만들어 "투표"를 하게 하는 투표 분류기를 만들어 봅시다
* `scikit-learn.ensemble.VotingClassifier`를 쓰시면 됩니다.

In [36]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
import seaborn as sns

* 먼저 데이터를 로드해 봅시다

In [37]:
X = np.load("W14/titanic_X_train.npy")
y = np.load("W14/titanic_y_train.npy")

In [38]:
X[0]

array([0.27345609, 0.01415106, 0.        , 1.        , 0.        ,
       0.125     , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [39]:
y[:10]

array([0., 1., 1., 1., 0., 0., 0., 0., 1., 1.])

* 모델을 생성하기 앞서 우리가 배운 가장 기본적인 모델 3가지(`LogisticRegression`, `DecisionTreeClassifier`, `GaussianNB`를 기본 모델로 삼습니다
* scikit-learn에선 이런 것들을 예측기 (estimator)라고 부릅니다
  * 리스트 형태로 `VotingClassifier` 의 인자로 넣을 수 있습니다

In [40]:
clf1 = LogisticRegression(random_state=1)
clf2 = DecisionTreeClassifier(random_state=1, max_depth=4)
clf3 = GaussianNB()

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('gnb',clf3)], voting='hard')

* 앙상블 모델의 성능을 측정해 봅시다.

In [41]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8222941661905668)

* 개별 모델들의 성능을 봅시다

In [42]:
# Logistic Regression
cross_val_score(clf1, X, y, cv=5).mean()

np.float64(0.8290420872214816)

In [43]:
# Decision Tree
cross_val_score(clf2, X, y, cv=5).mean()

np.float64(0.8223068621849807)

In [44]:
# Gaussian Naive Bayes
cross_val_score(clf3, X, y, cv=5).mean()

np.float64(0.4600139655938551)

* GNB가 성능이 좋지 않은 것을 알 수 있습니다. 
  * GNB는 연속 데이터에 잘 맞는 모델로, 이러한 이산 데이터에 잘 맞지 않을 수 있습니다
  * 이를 빼고 성능을 다시 측정해봅시다

In [35]:
eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2)], voting='hard')
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8301783787215135)

* 성능이 올랐습니다!
  * 투표 모델에서 꼭 많은 수의 모델을 쓴다고 더 좋은 성능을 내 주는 것은 아닙니다

### 하이퍼 파라미터를 튜닝한 투표 분류기
* 성능을 더 올리려면 하이퍼 매개변수를 튜닝해볼 수 있습니다. 

In [45]:
clf1 = LogisticRegression(random_state=1)
clf2 = DecisionTreeClassifier(random_state=1)
eclf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2)], voting='hard')

* 파라미터를 지정하고, `GridSearchCV`를 생성해서 모델을 만듭시다

In [46]:
c_params = [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]

params ={
    "lr__solver" :
        ['liblinear'], "lr__penalty" : ["l2"], "lr__C" : c_params,
    "dt__criterion" : ["gini", "entropy"],
    "dt__max_depth" : [10,8,7,6,5,4,3,2],
    "dt__min_samples_leaf": [1,2,3,4,5,6,7,8,9]
}

In [47]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X, y)
grid.best_score_

np.float64(0.8425569732749316)

* 아래의 상태에서 가장 좋은 성능을 내 주는 것을 알 수가 있습니다. 

In [48]:
grid.best_params_

{'dt__criterion': 'gini',
 'dt__max_depth': 10,
 'dt__min_samples_leaf': 5,
 'lr__C': 5.0,
 'lr__penalty': 'l2',
 'lr__solver': 'liblinear'}

## 배깅과 랜덤 포레스트

### 배깅
* 배깅은 하나의 데이터셋에서 샘플링으로 여러 데이터를 만든 다음 데이터마다 모델을 만들어서 투표분류기를 만드는 기법입니다. 
* 부트스트래핑이라는 기법을 씁니다
  * 모집단부터 학습 데이터를 추출할 때 복원추출을 반복합니다.

### 랜덤 포레스트
* 배깅에서 가장 많이 쓰이고 유명한 모델은 랜덤 포레스트 입니다
* 배깅을 의사결정트리에 적용한 모델입니다.

* 배깅은 scikit-learn에는 `sklearn.ensemble.BaggingClassifier`를 쓰시면 됩니다

* 주요 파라미터는 아래와 같습니다.
  * `base_estimator` : 사용될 수 있는 모델(default=None)
  * `n_estimators` : int, optional(default=10), subset으로 생성되는 모델의 개수
  * `max_samples` : int or float, optional(default=1.0), 최대 데이터 개수 또는 비율
  * `max_features` : int or float, optional(default=1.0), 최대 사용 피쳐 또는 비율
  * `bootstrap` : boolean, optional(default=True), bootstrap 사용 여부
  * `oob_score` : boolean, oob score 산출 여부
  * `warm_start` : booeanl, optional(default=False), 이전에 학습된 모델을 사용할 것인가에 대한 정보

* 일단 모델을 생성해 봅시다

In [50]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier

X = np.load("W14/titanic_X_train.npy")
y = np.load("W14/titanic_y_train.npy")
clf1 = LogisticRegression(random_state=1)
eclf = BaggingClassifier(clf1, oob_score=True, n_estimators=50) # Userwarning을 막기 위해 

from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8267948962102458)

In [51]:
params ={
    #"n_estimators" : [10,20,30,40,50,55],
    "n_estimators" : [40,50,55], # Userwarning을 막기 위해
    "max_samples" : [0.5,0.6,0.7,0.8,0.9,1]
}

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X, y)

grid.best_score_

# Warning이많이 발생합니다. 파라미터 문제...

np.float64(0.8312892782327175)

In [52]:
grid.best_params_

{'max_samples': 0.9, 'n_estimators': 55}

In [53]:
grid.best_estimator_.oob_score_

0.8323959505061868

* `sklearn.ensemble.RandomForestClassifier` 로 직접 구현된 버전도 있습니다.

In [54]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

X = np.load("W14/titanic_X_train.npy")
y = np.load("W14/titanic_y_train.npy")

eclf = RandomForestClassifier(n_estimators=100, max_features=2, n_jobs=7, oob_score=True)

from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.7975750650669714)

In [55]:
from sklearn.model_selection import GridSearchCV

params ={
    #"n_estimators" : [10, 20, 30, 50, 100],
    "n_estimators" : [30, 50, 100],
    "max_features" : [1,2,3,4,5,6,7, 10, 15, 20, 25, len(X[0])]
    }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X, y)

grid.best_score_

/home/jhyun/anaconda3/envs/ml2024/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


np.float64(0.8234495016822192)

In [56]:
grid.best_params_

{'max_features': 15, 'n_estimators': 50}

In [57]:
grid.best_estimator_.oob_score_

0.8200224971878515

### 부스팅
* 모델을 병렬로 만드는 배깅과 다르게, 부스팅은 틀렸던 답을 더 잘 맞추도록 다음 모델을 만들어 줍니다
* 마치 오답노트를 만드는 것과 같습니다.
* 병렬화가 어렵다는 단점이...
* 일단 모델을 만들어 봅시다

In [69]:
import numpy as np
X = np.load("W14/titanic_X_train.npy")
y = np.load("W14/titanic_y_train.npy")

* 에이다부스트에 트리 모델을 결합할 것입니다

In [81]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
#eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=500) # 교과서의 잘못된 코드입니다.
eclf = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), n_estimators=500, algorithm="SAMME") 

In [82]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8279248397130706)

In [83]:
from sklearn.ensemble import RandomForestClassifier
eclf = RandomForestClassifier(n_estimators=500)
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8020694470894434)

* `GridSearchCV`로 최적 모델을 찾아봅시다

In [84]:
# 아래에 해당하는 교과서 코드가 이상해서 수정을 좀 했습니다. 

eclf = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), n_estimators=500, algorithm="SAMME")

params = {"estimator__criterion" : ["gini", "entropy"],
          "estimator__max_features" : [7,8,],
          "estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
          }

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X, y)

grid.best_score_

np.float64(0.8222814701961532)

In [85]:
grid.best_params_

{'estimator__criterion': 'gini',
 'estimator__max_depth': 2,
 'estimator__max_features': 7,
 'learning_rate': 0.6,
 'n_estimators': 23}

In [86]:
grid.best_estimator_.feature_importances_

array([9.63652832e-02, 1.35372372e-01, 5.71357716e-02, 5.07146513e-02,
       1.03749370e-01, 6.73581065e-02, 9.23193977e-02, 6.87948885e-03,
       5.18089152e-16, 2.93754818e-02, 2.05661726e-03, 0.00000000e+00,
       0.00000000e+00, 2.14127000e-02, 0.00000000e+00, 1.99362475e-01,
       4.42392932e-05, 5.48274777e-02, 1.65561362e-02, 0.00000000e+00,
       2.59273511e-02, 3.49727429e-03, 6.71287560e-03, 2.37344376e-02,
       0.00000000e+00, 6.59849229e-03, 0.00000000e+00])

## Today
* 앙상블 모델
  * 투표 모델
  * 배깅 모델
    * 부트스트래핑은 무엇인가
  * 부스팅 모델

## Next class
- 시험... 시험을 봅시다.
- 종강 축하드립니다!